In [1]:
"""
This will add in the data from the leftovers of the excitatory meshes as well

Ideas on how to merge them:
1) Do the same process in a parallel manner for both left over and regular
2) At the end for all synapses:
a) Take the synapse with the closest label
b) Synapse exclude is just a union of both of the synapse exclude lists

"""

'\nThis will add in the data from the leftovers of the excitatory meshes as well\n\nIdeas on how to merge them:\n1) Do the same process in a parallel manner for both left over and regular\n2) At the end for all synapses:\na) Take the synapse with the closest label\nb) Synapse exclude is just a union of both of the synapse exclude lists\n\n'

# Synapse Labeling (with length threshold)

In [2]:
import datajoint as dj
import numpy as np
from scipy.spatial import KDTree
import time
from tqdm import tqdm
import ipyvolume.pylab as p3


In [3]:
dj.config['display.limit'] = 20
pinky = dj.create_virtual_module('pinky', 'microns_pinky')

Connecting celiib@10.28.0.34:3306


In [4]:
schema = dj.schema('microns_pinky')

@schema
class SynapseCompartmentLabelFinalThresholdRevised(dj.Manual):
    definition = """
    -> pinky.Synapse
    ---
    (postsynaptic_label)->pinky.LabelKey
    """ # (postsynaptic_label)->pinky.LabelKey : tinyint unsigned

In [5]:
#for the regular excitatory meshes
segment_ids = pinky.CoarseLabelFinal.fetch('segment_id')
mesh_fetch = (pinky.PymeshfixDecimatedExcitatoryStitchedMesh & pinky.CompartmentFinal.proj()).fetch()
label_fetch = pinky.CoarseLabelFinal.fetch()

pinky.PymeshfixDecimatedExcitatoryStitchedMesh & pinky.CompartmentFinal.proj()

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,n_vertices total number of vertices after pymeshfix cleaning,n_triangles total number of faces after pymeshfix cleaning,vertices vertices after pymeshfix cleaning,triangles faces after pymeshfix cleaning,perc_vertices_remaining number of vertices still present after pymeshfix cleaning,perc_triangles_remaining number of faces still present after pymeshfix cleaning
3,648518346341371119,0.35,223210,446728,=BLOB=,=BLOB=,0.99779,1.00152
3,648518346349386137,0.35,27998,56076,=BLOB=,=BLOB=,0.98842,0.99452
3,648518346349470171,0.35,352926,706364,=BLOB=,=BLOB=,0.91522,0.91810
3,648518346349471156,0.35,257594,515604,=BLOB=,=BLOB=,0.94457,0.94117
3,648518346349471500,0.35,232394,465152,=BLOB=,=BLOB=,0.90327,0.89243
3,648518346349471562,0.35,448173,896942,=BLOB=,=BLOB=,0.99005,0.99361
3,648518346349471565,0.35,762502,1525920,=BLOB=,=BLOB=,0.99868,1.00177
3,648518346349471910,0.35,659301,1320374,=BLOB=,=BLOB=,0.94923,0.94471
3,648518346349472574,0.35,524294,1049448,=BLOB=,=BLOB=,0.94572,0.93933
3,648518346349472601,0.35,514470,1029704,=BLOB=,=BLOB=,0.99875,1.00170


In [6]:
pinky.CoarseLabelFinal()

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,author_original name of last editor,author_proofreader name of last editor,date_time the last time it was edited,vertices Corrected vertex labels,triangles Same triangle labels as pinky.ProofreadLabel
3,648518346341371119,0.35,computer_Auto,celiib,2019-07-12 11:52:31,=BLOB=,=BLOB=
3,648518346349386137,0.35,computer_Auto,celiib,2019-06-11 18:53:38,=BLOB=,=BLOB=
3,648518346349470171,0.35,computer_Auto,celiib,2019-06-11 18:49:31,=BLOB=,=BLOB=
3,648518346349471156,0.35,computer_Auto,celiib,2019-06-12 13:53:20,=BLOB=,=BLOB=
3,648518346349471500,0.35,computer_Auto,celiib,2019-06-12 14:18:35,=BLOB=,=BLOB=
3,648518346349471562,0.35,computer_Auto,celiib,2019-07-12 11:47:49,=BLOB=,=BLOB=
3,648518346349471565,0.35,computer_Auto,celiib,2019-06-12 14:11:11,=BLOB=,=BLOB=
3,648518346349471910,0.35,computer_Auto,celiib,2019-07-12 11:44:28,=BLOB=,=BLOB=
3,648518346349472574,0.35,computer_Auto,celiib,2019-06-12 14:27:07,=BLOB=,=BLOB=
3,648518346349472601,0.35,computer_Auto,celiib,2019-07-12 11:42:05,=BLOB=,=BLOB=


In [7]:
#for the regular excitatory meshes
segment_ids = pinky.CoarseLabelFinal.fetch('segment_id')
mesh_fetch = (pinky.PymeshfixDecimatedExcitatoryStitchedMesh & pinky.CompartmentFinal.proj()).fetch()
label_fetch = pinky.CoarseLabelFinal.fetch()

meshes = dict()
labels = dict()
for segment in tqdm(mesh_fetch):
    meshes[segment[1]] = (segment[5], segment[6]) #maps the id number to the vertices and triangles
    
for segment in tqdm(label_fetch):
    labels[segment[1]] = (segment[6], segment[7]) #maps the id number to the vertices and triangle labels
    
kdtrees = dict()
for segment_id in tqdm(segment_ids):
    kdtrees[segment_id] = KDTree(meshes[segment_id][0]) #creates a KDTree from the mesh vertices

#gets all of the mesh synapses for that id
mesh_synapses = dict()
for segment_id in tqdm(segment_ids):
    mesh_synapses[segment_id] = np.array((pinky.Synapse & pinky.CurrentSegmentation
#                                          & dict(postsyn=segment_id)).fetch('synapse_id', 'presyn_x', 'presyn_y', 'presyn_z')).T
                                           & dict(postsyn=segment_id)).fetch('synapse_id', 'postsyn_x', 'postsyn_y', 'postsyn_z')).T

100%|██████████| 326/326 [01:35<00:00,  2.69it/s]


In [8]:
#for the regular excitatory meshes
segment_ids_leftover = (pinky.LeftoverCompartmentFinal.proj()).fetch('segment_id')
mesh_fetch_leftover = (pinky.ExcitatoryLeftoverMeshes & pinky.LeftoverCompartmentFinal.proj()).fetch()
label_fetch_leftover = (pinky.LeftoverCoarseLabelFinal & pinky.LeftoverCompartmentFinal.proj()).fetch()

meshes_leftover = dict()
labels_leftover = dict()
for segment in tqdm(mesh_fetch_leftover):
    meshes_leftover[segment[1]] = (segment[5], segment[6]) #maps the id number to the vertices and triangles
    
for segment in tqdm(label_fetch_leftover):
    labels_leftover[segment[1]] = (segment[6], segment[7]) #maps the id number to the vertices and triangle labels
    
kdtrees_leftover = dict()
for segment_id in tqdm(segment_ids_leftover):
    kdtrees_leftover[segment_id] = KDTree(meshes_leftover[segment_id][0]) #creates a KDTree from the mesh vertices

    
"""
Don't need to get the synapse
"""

100%|██████████| 320/320 [01:02<00:00,  1.07it/s]


"\nDon't need to get the synapse\n"

In [12]:
matched_synapses = dict()
synapses_to_exclude = []
for segment_id in tqdm(segment_ids):
    #gets the synapses corresponding to the segment
    synapses = mesh_synapses[segment_id] 
    #gets the ids and coordinates for those snapses
    synapse_ids, postsyn_coords = synapses.T[0], synapses.T[1:].T 
    #gets a KDTree for that segment based on its mesh
    kdtree = kdtrees[segment_id] 
    #gets the nearest node and distance for each synapse (have to adjust for the coordinates)
    distances, nearest_nodes = kdtree.query(postsyn_coords * [4, 4, 40])
    #gets the vertex labels
    vertex_labels = labels[segment_id][0]
    synapse_labels = dict()
    
    #check if the segment_id is the leftovers
    if segment_id not in segment_ids_leftover:
        for synapse_id, nearest_node, distance_node in tqdm(zip(synapse_ids, nearest_nodes,distances)):
            if distance_node > 5000:
                synapses_to_exclude.append(dict(segmentation=3,synapse_id=synapse_id))
            else:
                synapse_labels[synapse_id] = vertex_labels[nearest_node]
        matched_synapses[segment_id] = synapse_labels
    
    else:
        #gets a KDTree for that segment based on its mesh
        kdtree_leftover = kdtrees_leftover[segment_id] 
        #gets the nearest node and distance for each synapse (have to adjust for the coordinates)
        distances_leftover, nearest_nodes_leftover = kdtree_leftover.query(postsyn_coords * [4, 4, 40])
        #gets the vertex labels
        vertex_labels_leftover = labels_leftover[segment_id][0]
        


        for synapse_id, nearest_node, nearest_node_left,distance_node,distance_node_left in tqdm(zip(synapse_ids,
                                                                nearest_nodes,nearest_nodes_leftover,
                                                                distances,distances_leftover)):
            if distance_node > 5000 and distance_node_left > 5000:
                synapses_to_exclude.append(dict(segmentation=3,synapse_id=synapse_id))
            else:
                if distance_node < distance_node_left:
                    synapse_labels[synapse_id] = vertex_labels[nearest_node]
                else:
                    synapse_labels[synapse_id] = vertex_labels_leftover[nearest_node_left]
        matched_synapses[segment_id] = synapse_labels

  0%|          | 0/326 [00:00<?, ?it/s]
0it [00:00, ?it/s]
  0%|          | 1/326 [00:00<04:01,  1.35it/s]
0it [00:00, ?it/s]
49it [00:00, 62487.35it/s]
0it [00:00, ?it/s]
  1%|          | 3/326 [00:02<04:32,  1.18it/s]
0it [00:00, ?it/s]
  1%|          | 4/326 [00:03<04:31,  1.19it/s]
0it [00:00, ?it/s]
  2%|▏         | 5/326 [00:04<04:12,  1.27it/s]
0it [00:00, ?it/s]
  2%|▏         | 6/326 [00:06<06:13,  1.17s/it]
0it [00:00, ?it/s]
  2%|▏         | 7/326 [00:09<08:30,  1.60s/it]
0it [00:00, ?it/s]
  2%|▏         | 8/326 [00:12<10:42,  2.02s/it]
0it [00:00, ?it/s]
  3%|▎         | 9/326 [00:14<10:42,  2.03s/it]
0it [00:00, ?it/s]
  3%|▎         | 10/326 [00:15<10:02,  1.91s/it]
0it [00:00, ?it/s]
  3%|▎         | 11/326 [00:18<10:49,  2.06s/it]
0it [00:00, ?it/s]
  4%|▎         | 12/326 [00:21<12:15,  2.34s/it]
0it [00:00, ?it/s]
  4%|▍         | 13/326 [00:21<09:41,  1.86s/it]
0it [00:00, ?it/s]
  4%|▍         | 14/326 [00:24<11:18,  2.18s/it]
0it [00:00, ?it/s]
  5%|▍         | 15

In [11]:
nearest_node_left

214172

In [ ]:
#pinky.ExclusionCriteria.insert1(dict(criteria_id=0,criteria_desc='Labeling pipeline'))

In [13]:
pinky.ExclusionCriteria()

criteria_id key by which to lookup the exclusion criteria description,criteria_desc criteria by which this segment was excluded (preferred to add name of excluder as well)
0,Labeling pipeline
1,postsyn label was error in labeling pipeline
2,All error segments


In [14]:
for syn in synapses_to_exclude:
    syn["criteria_id"] = 0

In [15]:
pinky = dj.create_virtual_module('pinky', 'microns_pinky')

In [16]:
#add to synapse exclude
pinky.SynapseExclude.insert(synapses_to_exclude,skip_duplicates=True)

In [17]:
#need to insert the synapse excludes into the table

In [18]:
pinky.SynapseCompartmentLabelFinalThresholdRevised()

segmentation segmentation id,synapse_id synapse index within the segmentation,postsynaptic_label numeric label of the compartment
3,835,2
3,2426,2
3,3848,2
3,5106,2
3,7656,2
3,8128,2
3,8372,2
3,8472,2
3,9439,2
3,9673,2


In [19]:
segmentation = pinky.CurrentSegmentation.fetch1('segmentation')
all_synapse_labels = list()
for labeled_synapses in tqdm(matched_synapses.values()):
    segmentation_array = np.array([segmentation] * len(labeled_synapses))
    synapse_to_label = np.array(list(labeled_synapses.items())).T
    proper_label_array = np.array((segmentation_array, *synapse_to_label)).T
    all_synapse_labels.extend(proper_label_array)
all_synapse_labels_array = np.array(all_synapse_labels)

all_synapse_labels_array.T[2][all_synapse_labels_array.T[2]>12] = 1

start = time.time()
SynapseCompartmentLabelFinalThresholdRevised.insert(all_synapse_labels_array, skip_duplicates=True)
print(time.time() - start)

100%|██████████| 326/326 [00:00<00:00, 618.51it/s]


85.60616683959961


# Spine Synapse Labeler (with length threshold)

In [21]:
@schema
class SynapseSpineLabelFinalThresholdRevised(dj.Manual):
    definition = """
    # Denotes which synapses are also on Spine Heads, Spines, or Spine Necks
    -> pinky.SynapseCompartmentLabelFinalThresholdRevised
    ---
    (spine_label)->pinky.LabelKey
    """ # (spine_label)->pinky.LabelKey : tinyint unsigned

In [22]:
#gets the meshes
mesh_fetch = np.array((pinky.PymeshfixDecimatedExcitatoryStitchedMesh & pinky.CompartmentFinal.proj()).fetch('segment_id', 'vertices', 'triangles')).T
#gets the overlayed labels
label_fetch = np.array(pinky.OverlayedSpineLabel.fetch('segment_id', 'vertices', 'triangles')).T

meshes = dict()
#creates a dictionary with the mesh vertices and triangles for each segment id
for segment_id, mesh_vertices, mesh_triangles in tqdm(mesh_fetch):
    meshes[segment_id] = (mesh_vertices, mesh_triangles)

#creates a dictionary with the mesh vertices and triangles labels for each segment id
labels = dict()
for segment_id, label_vertices, label_triangles in tqdm(label_fetch):
    labels[segment_id] = (label_vertices, label_triangles)

##creates a dictionary with the kd tree for each segment id mesh
kdtrees = dict()
for segment_id in tqdm(segment_ids):
    kdtrees[segment_id] = KDTree(meshes[segment_id][0])
    
mesh_synapses = dict()
for segment_id in tqdm(segment_ids):
    mesh_synapses[segment_id] = np.array((pinky.Synapse & pinky.CurrentSegmentation
                                          & dict(postsyn=segment_id)).fetch('synapse_id', 'postsyn_x','postsyn_y', 'postsyn_z')).T

100%|██████████| 326/326 [01:35<00:00,  2.62it/s]


In [23]:
#gets the meshes
mesh_fetch_leftover = np.array((pinky.ExcitatoryLeftoverMeshes & pinky.LeftoverCompartmentFinal.proj()).fetch('segment_id', 'vertices', 'triangles')).T
#gets the overlayed labels
label_fetch_leftover = np.array(pinky.LeftoverOverlayedSpineLabel.fetch('segment_id', 'vertices', 'triangles')).T

meshes_leftover = dict()
#creates a dictionary with the mesh vertices and triangles for each segment id
for segment_id, mesh_vertices, mesh_triangles in tqdm(mesh_fetch_leftover):
    meshes_leftover[segment_id] = (mesh_vertices, mesh_triangles)

#creates a dictionary with the mesh vertices and triangles labels for each segment id
labels_leftover = dict()
for segment_id, label_vertices, label_triangles in tqdm(label_fetch_leftover):
    labels_leftover[segment_id] = (label_vertices, label_triangles)

##creates a dictionary with the kd tree for each segment id mesh
kdtrees_leftover = dict()
for segment_id in tqdm(segment_ids_leftover):
    kdtrees_leftover[segment_id] = KDTree(meshes_leftover[segment_id][0])


100%|██████████| 320/320 [01:02<00:00,  1.07it/s]


In [24]:
matched_synapses = dict()
synapses_to_exclude = []
for segment_id in tqdm(segment_ids):
    #gets the synapses corresponding to the segment
    synapses = mesh_synapses[segment_id] 
    #gets the ids and coordinates for those snapses
    synapse_ids, postsyn_coords = synapses.T[0], synapses.T[1:].T 
    #gets a KDTree for that segment based on its mesh
    kdtree = kdtrees[segment_id] 
    #gets the nearest node and distance for each synapse (have to adjust for the coordinates)
    distances, nearest_nodes = kdtree.query(postsyn_coords * [4, 4, 40])
    #gets the vertex labels
    vertex_labels = labels[segment_id][0]
    synapse_labels = dict()
    
    #check if the segment_id is the leftovers
    if segment_id not in segment_ids_leftover:
        for synapse_id, nearest_node, distance_node in tqdm(zip(synapse_ids, nearest_nodes,distances)):
            if distance_node > 5000:
                synapses_to_exclude.append(dict(segmentation=3,synapse_id=synapse_id))
            else:
                synapse_labels[synapse_id] = vertex_labels[nearest_node]
        matched_synapses[segment_id] = synapse_labels
    
    else:
        #gets a KDTree for that segment based on its mesh
        kdtree_leftover = kdtrees_leftover[segment_id] 
        #gets the nearest node and distance for each synapse (have to adjust for the coordinates)
        distances_leftover, nearest_nodes_leftover = kdtree_leftover.query(postsyn_coords * [4, 4, 40])
        #gets the vertex labels
        vertex_labels_leftover = labels_leftover[segment_id][0]
        


        for synapse_id, nearest_node, nearest_node_left,distance_node,distance_node_left in tqdm(zip(synapse_ids,
                                                                nearest_nodes,nearest_nodes_leftover,
                                                                distances,distances_leftover)):
            if distance_node > 5000 and distance_node_left > 5000:
                synapses_to_exclude.append(dict(segmentation=3,synapse_id=synapse_id))
            else:
                if distance_node < distance_node_left:
                    synapse_labels[synapse_id] = vertex_labels[nearest_node]
                else:
                    synapse_labels[synapse_id] = vertex_labels_leftover[nearest_node_left]
        matched_synapses[segment_id] = synapse_labels

  0%|          | 0/326 [00:00<?, ?it/s]
0it [00:00, ?it/s]
  0%|          | 1/326 [00:00<04:02,  1.34it/s]
0it [00:00, ?it/s]
49it [00:00, 58922.28it/s]
0it [00:00, ?it/s]
  1%|          | 3/326 [00:02<04:32,  1.19it/s]
0it [00:00, ?it/s]
  1%|          | 4/326 [00:03<04:26,  1.21it/s]
0it [00:00, ?it/s]
  2%|▏         | 5/326 [00:04<04:09,  1.29it/s]
0it [00:00, ?it/s]
  2%|▏         | 6/326 [00:06<06:07,  1.15s/it]
0it [00:00, ?it/s]
  2%|▏         | 7/326 [00:08<08:13,  1.55s/it]
0it [00:00, ?it/s]
  2%|▏         | 8/326 [00:11<10:15,  1.94s/it]
0it [00:00, ?it/s]
  3%|▎         | 9/326 [00:13<10:16,  1.95s/it]
0it [00:00, ?it/s]
  3%|▎         | 10/326 [00:15<09:43,  1.84s/it]
0it [00:00, ?it/s]
  3%|▎         | 11/326 [00:17<10:27,  1.99s/it]
0it [00:00, ?it/s]
  4%|▎         | 12/326 [00:20<11:50,  2.26s/it]
0it [00:00, ?it/s]
  4%|▍         | 13/326 [00:21<09:21,  1.79s/it]
0it [00:00, ?it/s]
  4%|▍         | 14/326 [00:24<11:01,  2.12s/it]
0it [00:00, ?it/s]
  5%|▍         | 15

In [26]:
for syn in synapses_to_exclude:
    syn["criteria_id"] = 0

In [27]:
len(pinky.SynapseExclude())

1151

In [28]:
len(pinky.SynapseExclude()) #use this one for the future

1151

In [29]:
# FIX THE SYNAPSES TO EXCLUDE SO HAS REASON

In [30]:
pinky.SynapseExclude.insert(synapses_to_exclude,skip_duplicates=True)


In [ ]:
segmentation = pinky.CurrentSegmentation.fetch1('segmentation')
all_synapse_labels = list()
for labeled_synapses in tqdm(matched_synapses.values()):
    segmentation_array = np.array([segmentation] * len(labeled_synapses))
    synapse_to_label = np.array(list(labeled_synapses.items())).T
    proper_label_array = np.array((segmentation_array, *synapse_to_label)).T
    all_synapse_labels.extend(proper_label_array)
all_synapse_labels_array = np.array(all_synapse_labels)

all_synapse_labels_array.T[2][all_synapse_labels_array.T[2]>15] = 1
all_synapse_labels_array = all_synapse_labels_array[all_synapse_labels_array.T[2]<=15]
all_synapse_labels_array = all_synapse_labels_array[all_synapse_labels_array.T[2]>12]

start = time.time()
SynapseSpineLabelFinalThresholdRevised.insert(all_synapse_labels_array, skip_duplicates=True)
print(time.time() - start)

100%|██████████| 326/326 [00:00<00:00, 734.62it/s]


## Visualization of Labeled Synapses (no labeled Spines yet)

In [ ]:
SynapseSpineLabelFinalThresholdRevised()

In [ ]:
# pinky.LabelKey()

In [ ]:
# pinky.ProofreadLabel & 'segment_id=648518346341393609'

In [ ]:
# label_keys = [( 0, 'not_labeled', 'no_color'), ( 1, 'label_removed', 'no_color'),
#                ( 2, 'Apical', 'blue'), ( 3, 'Basal', 'yellow'),
#                ( 4, 'Oblique', 'green'), ( 5, 'Soma', 'red'),
#                ( 6, 'Axon-Soma', 'aqua'), ( 7, 'Axon-Dendr', 'off blue'),
#                ( 8, 'Dendrite', 'purple'), ( 9, 'Distal', 'pink'),
#                (10, 'ignore Error', 'brown'), (11, 'Unlabelable', 'tan'),
#                (12, 'Cilia', 'orange'), (13, 'Spine Head (unsure if correct label)', 'red'),
#                (14, 'Spine (unsure if correct label)', 'blue'),
#                (15, 'Spine Neck (unsure if correct label)', 'green'),
#                (16, 'Bouton (unsure if correct label)', 'ignore')]
# segment_id = 648518346341393609
# synapse_label_types = np.unique(list(matched_synapses[segment_id].values()))
# label_matches = np.array(list(matched_synapses[segment_id].items())).T
# synapses_by_compartment = dict()
# synapse_centroids_by_compartment = dict()
# for synapse_label_type in synapse_label_types:
#     synapses_by_compartment[synapse_label_type] = label_matches[0][np.where(label_matches[1]==synapse_label_type)[0]]
#     centroids = np.zeros((len(synapses_by_compartment[synapse_label_type]), 3), dtype=np.uint32)
#     for i, specific_synapse in enumerate(synapses_by_compartment[synapse_label_type]):
#         centroids[i] = mesh_synapses[segment_id].T[1:].T[mesh_synapses[segment_id].T[0]==specific_synapse]
#     synapse_centroids_by_compartment[synapse_label_type] = centroids
# p3.figure(width=1024, height=1024)
# vertices, triangles = meshes[segment_id]
# p3.plot_trisurf(*vertices.T/1000, triangles=triangles, color='black')
# for label_type, label_name, label_color in label_keys:
#     if label_type in synapse_label_types:
#         print("{} -> {} : {}".format(label_type, label_name, label_color))
#         p3.scatter(*(synapse_centroids_by_compartment[label_type] * [4, 4, 40]).T/1000, color=label_color, marker='sphere', size=0.1)
# p3.squarelim()
# p3.show()